In [1]:
import pandas as pd
import os
import numpy as np
import xlsxwriter as xw
import re
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

# Date to split the files result

In [2]:
date_list = [['5-Aug-2022','12-Aug-2022', '19-Aug-2022', '26-Aug-2022', '2-Sep-2022', '9-Sep-2022',
             '16-Sep-2022', '23-Sep-2022', '28-Sep-2022', '30-Sep-2022'], ['6-Oct-2022'], ['7-Oct-2022'],
             ['14-Oct-2022'],['21-Oct-2022'], ['28-Oct-2022'], ['4-Nov-2022'], ['11-Nov-2022']]
type(date_list)

list

In [3]:
latest_data_set_date = "11-Nov-2022"

In [4]:
Default_Ingestion_Date = "8-Jun-2022"

# Location variable

In [5]:
input_loc = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 2 Google Dynamic Revisit/Target/'
output_loc = '/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 2 Google Dynamic Revisit/Result/Final/'

# Extract Data from Dynamics

In [6]:
Dynamic_source = pd.read_csv(
    "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 1 Invalid file read for dynamics/Result/Dynamic_source.csv")

In [7]:
Dynamic_source.shape

(2278167, 19)

In [8]:
Dynamic_source.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,RWM Revisit Status,RWM Revisit Date,Revisit Expire Date,Validity,Pincode,Phone Number,City,Cluster,Failed Upload RWM MID,csv Google MID,Google csv Visit Date,csv Validity,s no
0,BCR2DN4TSCO3DNDR,checkin-nob,ADD,5-Aug-2022,Success,5-Aug-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# Extracting from latest file
# Update the update date in the old records

In [9]:
Dynamic_source_f = Dynamic_source[['MerchantId', 'Objectives' ,'IngestionStatus', 'Report Date', 'RWM Remarks', 
                                   'Ingestion Date', 'Revisit Expire Date', 's no']]

In [10]:
Dynamic_source_f.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,Revisit Expire Date,s no
0,BCR2DN4TSCO3DNDR,checkin-nob,ADD,5-Aug-2022,Success,5-Aug-2022,NaN,1


In [11]:
Dynamic_source_f['Report Date'].unique()

array(['5-Aug-2022', '12-Aug-2022', '19-Aug-2022', '26-Aug-2022',
       '2-Sep-2022', '9-Sep-2022', '16-Sep-2022', '23-Sep-2022',
       '28-Sep-2022', '30-Sep-2022', '6-Oct-2022', '7-Oct-2022',
       '14-Oct-2022', '21-Oct-2022', '28-Oct-2022', '4-Nov-2022',
       '11-Nov-2022'], dtype=object)

In [12]:
Dynamic_source_final = Dynamic_source_f

### Dynamic Failed

In [13]:
Dynamic_source_failed = Dynamic_source_f[~Dynamic_source_f['RWM Remarks'].isin(['Success'])]

In [14]:
Dynamic_source_failed['RWM Remarks'].unique()

array(['Merchant Ext.ID is not present in Merchant Master.',
       'Merchant Ext. ID is not present in the Revisit Merchant Table.',
       'This Merchant Ext.ID is not present in Revisit Merchant.',
       'Duplicate Merchant Ext.ID Already Exist in Excel'], dtype=object)

In [15]:
Dynamic_source_failed = Dynamic_source_failed.drop_duplicates(subset=['MerchantId'], keep='last')

### Dynamic Success

In [16]:
Dynamic_source_f = Dynamic_source_f[Dynamic_source_f['RWM Remarks'].isin(['Success'])]

In [17]:
Dynamic_source_f['RWM Remarks'].unique()

array(['Success'], dtype=object)

In [18]:
Dynamic_source_f = Dynamic_source_f.drop_duplicates(subset=['MerchantId'], keep='last')

# Latest file

In [19]:
Dynamic_source.shape

(2278167, 19)

In [20]:
latest_file = Dynamic_source_f[Dynamic_source_f['Report Date'].isin([latest_data_set_date])]

In [21]:
latest_file['Report Date'].unique()

array(['11-Nov-2022'], dtype=object)

In [22]:
latest_file.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,Revisit Expire Date,s no
1991413,BCR2DN4T7T4YPRDM,Inactive - INR 40 Extra Pay on Txn,ADD,11-Nov-2022,Success,11-Nov-2022,NaN,1991414


In [23]:
latest_file.shape

(246424, 8)

# Adding primary key column in the Dynamics source

In [24]:
Dynamic_source_f['Primary_key'] = range(1,(len(Dynamic_source_f)+1))

### Filter out the latest dataset from the dynamics

In [25]:
Dynamic_source_f['Report Date'].unique()

array(['5-Aug-2022', '12-Aug-2022', '19-Aug-2022', '26-Aug-2022',
       '2-Sep-2022', '9-Sep-2022', '16-Sep-2022', '23-Sep-2022',
       '28-Sep-2022', '30-Sep-2022', '6-Oct-2022', '7-Oct-2022',
       '14-Oct-2022', '21-Oct-2022', '28-Oct-2022', '4-Nov-2022',
       '11-Nov-2022'], dtype=object)

In [26]:
Dynamic_source_f2 = Dynamic_source_f[~Dynamic_source_f['Report Date'].isin([latest_data_set_date])]

In [27]:
Dynamic_source_f = Dynamic_source_f2

In [28]:
Dynamic_source_old_record = Dynamic_source_f

In [29]:
Dynamic_source_old_record['Report Date'].unique()

array(['5-Aug-2022', '12-Aug-2022', '19-Aug-2022', '26-Aug-2022',
       '2-Sep-2022', '9-Sep-2022', '16-Sep-2022', '23-Sep-2022',
       '28-Sep-2022', '30-Sep-2022', '6-Oct-2022', '7-Oct-2022',
       '14-Oct-2022', '21-Oct-2022', '28-Oct-2022', '4-Nov-2022'],
      dtype=object)

In [30]:
Dynamic_source_old_record.shape

(1175459, 9)

In [31]:
Dynamic_source_old_record.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,Revisit Expire Date,s no,Primary_key
0,BCR2DN4TSCO3DNDR,checkin-nob,ADD,5-Aug-2022,Success,5-Aug-2022,NaN,1,1


### Filter  the latest dataset from the latest file

In [32]:
latest_file['Report Date'].unique()

array(['11-Nov-2022'], dtype=object)

In [33]:
latest_file_f = latest_file[latest_file['Report Date'].isin([latest_data_set_date])]

In [34]:
latest_file_f.shape

(246424, 8)

# Updating the old Dataset

### Updating the Latest Expiry Date

In [35]:
latest_file_f_d = latest_file_f[latest_file_f["IngestionStatus"].isin(['DELETE'])] 

In [36]:
latest_file_f_d2 = latest_file_f_d[['MerchantId', 'Revisit Expire Date']].rename(
    columns={'Revisit Expire Date':"latest_expire"})

In [37]:
Dynamic_source_old_record_merge = Dynamic_source_old_record.merge(latest_file_f_d2, how='left', on='MerchantId')

In [38]:
Dynamic_source_old_record_merge.drop_duplicates(subset=['Primary_key'], inplace=True)

In [39]:
Dynamic_source_old_record_merge['Revisit Expire Date'].fillna(Dynamic_source_old_record_merge[
    'latest_expire'], inplace=True)

### Updating the old objective in the latest objective

In [40]:
Dynamic_source_old_record_f = Dynamic_source_old_record[['MerchantId', 'Objectives']].rename(
    columns={'Objectives':"Old_Success_Objectives"})

In [41]:
latest_file_f_obj_suc = latest_file_f.merge(Dynamic_source_old_record_f, on='MerchantId', how='left')

In [42]:
latest_file_f_obj_suc.drop_duplicates(subset=['s no'], inplace=True)

In [43]:
Dynamic_source_failed = Dynamic_source_failed[~Dynamic_source_failed['Report Date'].isin([latest_data_set_date])]

In [44]:
Dynamic_source_failed_f = Dynamic_source_failed[['MerchantId', 'Objectives']].rename(
    columns={'Objectives':"Old_Failed_Objectives"})

In [45]:
latest_file_f_obj_suc_fail = latest_file_f_obj_suc.merge(Dynamic_source_failed_f, on='MerchantId', how='left')

In [46]:
latest_file_f_obj_suc_fail.drop_duplicates(subset=['s no'], inplace=True)

In [47]:
latest_file_f_obj_suc_fail['Objectives'].fillna(latest_file_f_obj_suc_fail['Old_Success_Objectives'], inplace=True)
latest_file_f_obj_suc_fail['Objectives'].fillna(latest_file_f_obj_suc_fail['Old_Failed_Objectives'], inplace=True)

In [48]:
latest_file_f_obj_suc_fail.head(1)

,MerchantId,Objectives,IngestionStatus,Report Date,RWM Remarks,Ingestion Date,Revisit Expire Date,s no,Old_Success_Objectives,Old_Failed_Objectives
0,BCR2DN4T7T4YPRDM,Inactive - INR 40 Extra Pay on Txn,ADD,11-Nov-2022,Success,11-Nov-2022,NaN,1991414,NaN,NaN


In [49]:
latest_file_f_obj_suc_fail_f = latest_file_f_obj_suc_fail[['s no', 'Objectives']].rename(
    columns={'Objectives':'Objectives_Delete'})

# Updating the ingestion date from the previous record

In [50]:
Dynamic_source_old_record_f = Dynamic_source_old_record[Dynamic_source_old_record['IngestionStatus'].isin([
    "ADD",'EDIT'])]

In [51]:
Dynamic_source_old_record_f2 = Dynamic_source_old_record_f[['MerchantId', 'Report Date']].rename(
    columns={'Report Date':"Old_Ingestion Date"})

In [52]:
latest_file_f_obj_suc_fail_inda = latest_file_f_obj_suc_fail.merge(
    Dynamic_source_old_record_f2, on='MerchantId', how='left')

In [53]:
latest_file_f_obj_suc_fail_inda.drop_duplicates(subset=['s no'], inplace=True)

In [54]:
latest_file_f_obj_suc_fail_inda['Old_Ingestion Date'].fillna(latest_file_f_obj_suc_fail_inda[
    'Ingestion Date'], inplace=True)

In [55]:
latest_file_f_obj_suc_fail_inda.drop(['Ingestion Date'], axis=1, inplace=True)

In [56]:
latest_file_f_obj_suc_fail_inda.rename(columns={'Old_Ingestion Date':'Ingestion Date'}, inplace=True)

In [57]:
latest_file_f_obj_suc_fail_inda['Ingestion Date'].fillna(Default_Ingestion_Date, inplace=True)

In [58]:
latest_file_f_obj_suc_fail_inda_f = latest_file_f_obj_suc_fail_inda[['s no', 'Ingestion Date', "Objectives"]]

In [59]:
latest_file_f_obj_suc_fail_inda_f.rename(columns={'Ingestion Date':"New Ingestion Date"}, inplace=True)
latest_file_f_obj_suc_fail_inda_f.rename(columns={'Objectives':"New Objectives"}, inplace=True)

In [60]:
Dynamic_source_merge = Dynamic_source.merge(latest_file_f_obj_suc_fail_inda_f, on='s no', how='left')

In [61]:
Dynamic_source_merge.drop_duplicates(subset=['s no'], inplace=True)

In [62]:
Dynamic_source_merge['Ingestion Date'].fillna(Dynamic_source_merge['New Ingestion Date'], inplace=True)

In [63]:
Dynamic_source_merge['Objectives'].fillna(Dynamic_source_merge['New Objectives'], inplace=True)

In [64]:
Dynamic_source_merge2 = Dynamic_source_merge.merge(latest_file_f_obj_suc_fail_f, on='s no', how='left')

In [65]:
Dynamic_source_merge2.drop_duplicates(subset=['s no'], inplace=True)

In [66]:
Dynamic_source_merge2['Objectives'].fillna(Dynamic_source_merge2['Objectives_Delete'], inplace=True)

In [67]:
Dynamic_source_merge2_f = Dynamic_source_merge2[['MerchantId', 'Objectives', 'IngestionStatus', 'Report Date',
                    'RWM Remarks', 'Ingestion Date', 'RWM Revisit Status', 'RWM Revisit Date', 
'Revisit Expire Date', 'Validity', 'Pincode', 'Phone Number', 'City', 'Cluster', 'Failed Upload RWM MID',
'csv Google MID', 'Google csv Visit Date', 'csv Validity', 's no']]

In [68]:
Dynamic_source_merge2_f.to_csv(output_loc+"Dynamic_Source.csv", index=False)

In [69]:
Dynamic_source_merge2_f2 = Dynamic_source_merge2_f[['MerchantId', 'Objectives', 'IngestionStatus', 'Report Date',
                    'RWM Remarks', 'Ingestion Date', 'Revisit Expire Date', 's no']]

In [70]:
Dynamic_source_merge2_f3 = Dynamic_source_merge2_f2[Dynamic_source_merge2_f2['RWM Remarks'].isin(['Success'])]

In [71]:
Dynamic_source_merge2_f3.drop_duplicates(subset=['MerchantId'], keep='last', inplace=True)

In [72]:
Dynamic_source_merge2_f4 = Dynamic_source_merge2_f3[['s no', 'IngestionStatus', 'Revisit Expire Date']]

In [73]:
Dynamic_source_merge2_f5 = Dynamic_source_merge2_f4[Dynamic_source_merge2_f4['IngestionStatus'].isin([
    'ADD', 'EDIT'])]

In [74]:
Dynamic_source_merge2_f5['Check_Expire'] = Dynamic_source_merge2_f5['Revisit Expire Date'].isna()

In [75]:
Dynamic_source_merge2_f5['Check_Expire'].replace(True, np.nan, inplace=True)

In [76]:
Dynamic_source_merge2_f6 = Dynamic_source_merge2_f5[['s no', 'Check_Expire']]

In [77]:
Dynamic_source_merge3_f3 = Dynamic_source_merge2_f3.merge(Dynamic_source_merge2_f6, on='s no', how='left')

In [78]:
Dynamic_source_merge3_f3['Check_Expire'].fillna(Dynamic_source_merge3_f3['Revisit Expire Date'], inplace=True)

In [79]:
Dynamic_source_merge3_f3['Check_Expire'].replace(False, np.nan, inplace=True)

In [80]:
Dynamic_source_merge3_f3.drop(['Revisit Expire Date'], axis=1, inplace=True)

In [81]:
Dynamic_source_merge3_f3.rename(columns={'Check_Expire':"Revisit Expire Date"}, inplace=True)

In [82]:
Dynamic_source_merge3_f3[['MerchantId', 'Objectives','RWM Remarks','IngestionStatus', 'Report Date', 'Ingestion Date',
                         'Revisit Expire Date', 's no']].to_csv(output_loc+"Dynamic_for_matching.csv", index=False)

In [85]:
Dynamic_source_merge3_f3['Report Date'].unique()

array(['5-Aug-2022', '12-Aug-2022', '19-Aug-2022', '26-Aug-2022',
       '2-Sep-2022', '9-Sep-2022', '16-Sep-2022', '23-Sep-2022',
       '28-Sep-2022', '30-Sep-2022', '6-Oct-2022', '7-Oct-2022',
       '14-Oct-2022', '21-Oct-2022', '28-Oct-2022', '4-Nov-2022',
       '11-Nov-2022'], dtype=object)

In [84]:
Stop = 

SyntaxError: invalid syntax (2587651117.py, line 1)

# Extra

In [86]:
output_loc_2 = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Script 1 Invalid file read for dynamics/Result/"

In [87]:
Dynamic_source_merge2_f2 = Dynamic_source_merge2_f[Dynamic_source_merge2_f['Report Date'].isin(['21-Oct-2022',
"28-Oct-2022", '4-Nov-2022', '11-Nov-2022'])]

In [88]:
Dynamic_source_merge2_f3 = Dynamic_source_merge2_f2[Dynamic_source_merge2_f2[
    'IngestionStatus'].isin(['ADD', 'EDIT'])]

In [89]:
Dynamic_source_merge2_f4 = Dynamic_source_merge2_f3[~Dynamic_source_merge2_f3['RWM Remarks'].isin(['Success'])]

In [90]:
Dynamic_source_merge2_f5 = Dynamic_source_merge2_f4[['MerchantId', 'Objectives', 'IngestionStatus','Report Date',
                                                    'RWM Remarks']]

In [91]:
Dynamic_ADD_Edit = Dynamic_source_merge2_f5

In [92]:
Dynamic_ADD_Edit.to_csv(output_loc_2+'Reingest/ADD_EDIT.csv', index=False)

In [93]:
Dynamic_source_del = Dynamic_source_merge2_f2[Dynamic_source_merge2_f2['IngestionStatus'].isin(['DELETE'])]

In [94]:
#Dynamic_source_del_success = Dynamic_source_del[Dynamic_source_del['RWM Remarks'].isin(['Success'])]

In [97]:
Dynamic_source_del_success_f = Dynamic_source_del[['MerchantId', 
'Objectives', 'IngestionStatus', 'Report Date', 'RWM Remarks']]

In [98]:
Dynamic_source_del_success_f.to_csv(output_loc_2+'Reingest/DELETE.csv', index=False)